# Lab 6 - Elementy Programowania Funkcyjnego

## Podstawy

Program składa się z **czystych funkcji**, czyli funkcji bez **efektów ubocznych**.

- Konstrukty odnoszą się do pojedynczych obiektów lub pojedynczej kolekcji obiektów
- Wykorzystywanie funkcji jako parametrów funkcji
- Rzadkie wykorzysywanie pętli
- Wykorzystanie `filter`, `map`, `reduce`
- Lambdy

### Podstawowe cechy

- podstawowa składnia funkcji `([parameters]) -> [result-type]`

- funkcja jest typem pierwszoklasowym - jest konstruktem służącym do przechowywania danych, na którym możemy wykonywać takie same operacje, jak na innych, wbudowanych typach

In [3]:
val f: (Int, String) -> String = { i:Int, s:String -> "${i}: ${s}" } // function literal
val f2 = { i:Int, s:String -> "${i}: ${s}" } // anonymous lambda

fun ff(fun1: (Int,String) -> String):String {
 return fun1(7, "Hello")
}
ff(f)

7: Hello

In [25]:
ff( { i:Int,s:String -> "${i}- ${s}" } )

7- Hello

- wywołanie ` function({ [lambda-function] })` może zostać skócone do `function { [lambda-function] }`

In [59]:
ff { i:Int,s:String -> "${i}- ${s}" }

7- Hello

In [5]:
object X {
 fun add(a:Int, b:Int): Int = a + b
}

val f : (Int,Int) -> Int = X::add

In [6]:
X.add(2, 2)

4

In [7]:
f(2, 3)

5

- odniesienie do funkcji z klas i obiektów przez `::`

In [26]:
class X2 {
 fun add(a:Int, b:Int): Int = a + b
}

val f : X2.(Int,Int) -> Int = X2::add
f(2, 2)

4

- odniesienie do metod instancji przez `::`

In [31]:
class X3 {
 fun add(a:Int, b:Int): Int = a + b
}

val x3 = X3()
val f : (Int,Int) -> Int = x3::add
f(2, 2)

4

In [32]:
val f : (Int,Int) -> Int = X3::add

Line_31.jupyter-kts (1:28 - 35) Type mismatch: inferred type is KFunction3<Line_30.X3, Int, Int, Int> but (Int, Int) -> Int was expected
Line_31.jupyter-kts (1:32 - 35) Type mismatch: inferred type is (Int, Int) -> Int but KFunction3<Line_30.X3, Int, Int, Int> was expected

In [33]:
X3.add(2, 2)

Line_32.jupyter-kts (1:4 - 7) Unresolved reference: add

In [34]:
x3.add(2, 2)

4

## Pętle

Pętle można zrealizować przez zastosowanie rekursji z funkcją pomocniczą. Typowo nazywa się `go` lub `loop`. Definicja `factorial` zawiera tylko wywołanie `go` z początkowymi warunkami pętli.

In [6]:
fun factorial(i: Int): Int {
    fun go(n: Int, acc: Int): Int =
        if (n <= 0) acc
        else go(n - 1, n * acc)
    return go(i, 1)
}

factorial(9)

362880

In [9]:
fun factorial2(i: Int): Int {
    tailrec fun go(n: Int, acc: Int): Int =
        if (n <= 0) acc
        else go(n - 1, n * acc)
    return go(i, 1)
}

factorial2(9)

362880

Przyjmuje się, że wywołanie znajduje się na `tail`, jeśli wywołujący nie robi nic poza zwrotem wartości rekurencyjnego wywołania.<br>
Przykładowo:<br>
- `go(n-1, n*acc)` jest w pozycji `tail` ponieważ metoda zwraca wartość wywołania i nic z nim więcej nie robi
- `1 + go(n-1, n*acc)` nie jest w pozycji `tail` ponieważ po zwróceniu `go` dalej wykonywana jest operacja dodawania<br>
Jeżeli wszystkie wywołania rekursywne są w pozycji `tail` zastosowanie modyfikatora `tailrec` powoduje zastąpienie rekursji wywołaniami iteratywnymi przez kompilator. Dzięki temu unikamy `StackOverflowError`.

## Funkcje wyższego rzędu

In [96]:
object Example {
    private fun abs(n: Int): Int =
        if (n < 0) -n
        else n
    
    private fun factorial(i: Int): Int {
        fun go(n: Int, acc: Int): Int =
            if (n <= 0) acc
            else go(n - 1, n * acc)
        return go(i, 1)
}
    
    fun formatAbs(x: Int): String {
        val msg = "The absolute value of %d is %d"
        return msg.format(x, abs(x))
    }

    fun formatFactorial(x: Int): String {
        val msg = "The factorial of %d is %d"
        return msg.format(x, factorial(x))
    }
}

fun main() {
    println(Example.formatAbs(-42))
    println(Example.formatFactorial(7))
}

main()

The absolute value of -42 is 42
The factorial of 7 is 5040


wprowadzamy funkcję wyższego rzędu

In [11]:
fun formatResult(name: String, n: Int, f: (Int) -> Int): String {
    val msg = "The %s of %d is %d."
    return msg.format(name, n, f(n))
}

In [12]:
fun main() {
    println(formatResult("factorial", 7, ::factorial))
    println(formatResult("absolute value", -42, ::abs))
}
main()

The factorial of 7 is 5040.
The absolute value of -42 is 42.


In [104]:
formatResult(
    "absolute", 
    -42,
    fun(n: Int): Int { return if (n < 0) -n else n } // funkcja anonimowa
)

The absolute of -42 is 42.

In [105]:
formatResult(
    "absolute", 
    -42, 
    { n -> if (n < 0) -n else n } // lambda anonimowa
)

The absolute of -42 is 42.

In [13]:
formatResult("absolute", -42) 
    { if (it < 0) -it else it }

The absolute of -42 is 42.

## Funkcje Monomorficzne i Polimorficzne

Funkcja `factorial` jest funkcją monomorficzną, przyjmuje tylko jeden typ argumentu.

Poniżej funkcja **monomorficzna** `findFirst` zwracająca indeks pierwszego wystąpienia elementu w tabeli `String`.

In [39]:
fun findFirstMonomorphic(ss: Array<String>, key: String): Int {
    tailrec fun loop(n: Int): Int =
        when {
            n >= ss.size -> -1
            ss[n] == key -> n
            else -> loop(n + 1)
        }
    return loop(0)
}

val a = arrayOf("Rafał", "Robert", "Robert")
val s = "Robert"
findFirstMonomorphic(a, s)

1

In [40]:
val a = arrayOf(1, 2, 3, 4)
val s = 2
findFirstMonomorphic(a, s)

Line_39.jupyter-kts (3:22 - 23) Type mismatch: inferred type is Array<Int> but Array<String> was expected
Line_39.jupyter-kts (3:25 - 26) Type mismatch: inferred type is Int but String was expected

Taką funkcję możemy zmienić na funkcję **polimorficzną** pracującą na argumencie typowalnym.

In [10]:
fun <A> findFirstPolimorphic(xs: Array<A>, p: (A) -> Boolean): Int 
{
    tailrec fun loop(n: Int): Int =
        when {
            n >= xs.size -> -1
            p(xs[n]) -> n
            else -> loop(n + 1)
    }
        
    return loop(0)
}

val a = arrayOf("Rafał", "Robert", "Robert", "Paweł")
findFirstPolimorphic(a) { it == "Paweł"}

3

In [15]:
findFirstPolimorphic(arrayOf(1, 2, 3, 4)){it == 4}

3

Funkcja **polimorficzna** jest funkcją **generyczną**.